In [1]:
from vcub_keeper.production.data import (get_data_from_api_by_station,
                                         transform_json_station_data_to_df)
from vcub_keeper.visualisation import *
from vcub_keeper.ml.cluster import train_cluster_station, predict_anomalies_station
from vcub_keeper.transform.features_factory import *
from vcub_keeper.ml.cluster_utils import load_model

%load_ext autoreload
%autoreload 2

pd.options.display.max_rows = 500

## Objectifs

- Faire des prédictions d'anomalies sur une station via les données de production

## Prédiction sur une station

In [20]:
station_id=124
start_date='2020-10-16'
#start_date='2020-09-01'
stop_date='2020-10-24'

In [21]:
station_json = get_data_from_api_by_station(station_id=station_id, 
                                            start_date=start_date,
                                            stop_date=stop_date)

station_df = transform_json_station_data_to_df(station_json)

KeyError: 'data'

In [16]:
station_df = get_consecutive_no_transactions_out(station_df) 

In [17]:
# Load clf 
clf = load_model(station_id=station_id)

In [18]:
# Faire une prédiction d'une station
station_pred = predict_anomalies_station(data=station_df, clf=clf, station_id=station_id)

In [19]:
plot_station_anomalies(data=station_df, clf=clf, station_id=station_id)

In [8]:
from vcub_keeper.reader.reader import *

station_profile = read_station_profile()

THRESHOLD_PROFILE_STATION = 0.3
station_profile[station_profile.station_id == station_id]

,station_id,total_point,mean,median,std,95%,98%,99%,max,profile_station_activity
180,39,64654,0.82,0.0,1.4,3.0,5.0,6.0,31.0,very high


In [9]:
station_profile

,station_id,total_point,mean,median,std,95%,98%,99%,max,profile_station_activity
0,181,2,0.00,0.0,0.00,0.0,0.0,0.00,0.0,low
1,160,64496,0.00,0.0,0.07,0.0,0.0,0.00,5.0,low
2,161,64028,0.01,0.0,0.10,0.0,0.0,0.00,6.0,low
3,180,31672,0.01,0.0,0.12,0.0,0.0,0.00,8.0,low
4,92,65219,0.01,0.0,0.12,0.0,0.0,0.00,6.0,low
5,167,62437,0.01,0.0,0.14,0.0,0.0,0.00,9.0,low
6,168,50066,0.01,0.0,0.13,0.0,0.0,1.00,6.0,low
7,183,17277,0.01,0.0,0.14,0.0,0.0,1.00,5.0,low
8,81,65125,0.02,0.0,0.16,0.0,0.0,1.00,8.0,low
9,150,64702,0.02,0.0,0.16,0.0,0.0,1.00,9.0,low


## Prediction sur toutes les stations

In [16]:
from datetime import timedelta, date


station_id = 102
date_today = date.today()
end_date = date_today + timedelta(days=1)
start_date = date_today - timedelta(days=8)

start_date_str = start_date.strftime('%Y-%m-%d')
end_date_str = end_date.strftime('%Y-%m-%d')

#start_date_str='2020-10-16'
#end_date_str='2020-10-24'

In [17]:
start_date_str

'2020-10-20'

In [18]:
from vcub_keeper.reader.reader import *
from vcub_keeper.config import THRESHOLD_PROFILE_STATION

#THRESHOLD_PROFILE_STATION = 0.3

station_profile = read_station_profile()

stations_id_to_pred = \
    station_profile[station_profile['mean'] >= THRESHOLD_PROFILE_STATION]['station_id'].unique()

In [22]:
station_json = get_data_from_api_by_station(station_id=stations_id_to_pred, #stations_id_to_pred, station_id
                                        start_date=start_date_str,
                                        stop_date=end_date_str)

station_df = transform_json_station_data_to_df(station_json)

station_df = get_consecutive_no_transactions_out(station_df) 
    
for station_id in stations_id_to_pred:    
    # Load clf 
    clf = load_model(station_id=station_id)

    # Faire une prédiction d'une station
    station_pred = predict_anomalies_station(data=station_df, clf=clf, station_id=station_id)

    if (station_pred.tail(10)['anomaly'] == -1).any():
        print('Recent Anomly detected for station N° ' + str(station_id))
    elif (station_pred.tail(1)['status'] == 0).any():
        print('Station N° ' + str(station_id) + " is inactive")
    else:
        print('Clean for station N°' + str(station_id))

KeyError: 'data'

In [9]:
station_pred.tail()

,station_id,date,available_stands,available_bikes,status,transactions_in,transactions_out,transactions_all,consecutive_no_transactions_out,weekday,hours,Sin_weekday,Cos_weekday,Sin_hours,Cos_hours,anomaly
58459,106,2020-10-28 10:20:00,17.0,23.0,1.0,2.0,0.0,2.0,1,2,10,0.974928,-0.222521,0.500000,-0.866025,1
58460,106,2020-10-28 10:30:00,18.0,22.0,1.0,0.0,1.0,1.0,0,2,10,0.974928,-0.222521,0.500000,-0.866025,1
58461,106,2020-10-28 10:40:00,16.0,24.0,1.0,2.0,0.0,2.0,1,2,10,0.974928,-0.222521,0.500000,-0.866025,1
58462,106,2020-10-28 10:50:00,17.0,23.0,1.0,0.0,1.0,1.0,0,2,10,0.974928,-0.222521,0.500000,-0.866025,1
58463,106,2020-10-28 11:00:00,15.0,25.0,1.0,2.0,0.0,2.0,1,2,11,0.974928,-0.222521,0.258819,-0.965926,1
